**Variables de entorno**

In [2]:
import dotenv
import sys
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import openai
openai.api_type = os.environ["OPENAI_API_TYPE"]
openai.api_version = os.environ["OPENAI_API_VERSION"]
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_key = os.environ["OPENAI_API_KEY"]

### **Vista general de los LLM**

Los LLM son modelos de aprendizaje profundo (deep learning) con miles de millones de parámetros que destacan en una amplia gama de tareas de procesamiento del lenguaje natural. Pueden realizar tareas como la traducción, el análisis de sentimientos y las conversaciones de chatbot sin haber sido entrenados específicamente para ello. Los LLM pueden utilizarse sin necesidad de un fine-tuning empleando técnicas de "prompting", en las que una pregunta se presenta como un prompt de texto con ejemplos de problemas y soluciones similares.

- Arquitectura:

Los LLM constan de varias capas de redes neuronales, capas feedforward, capas de embeddings y capas de atención. Estas capas trabajan juntas para procesar el texto de entrada y generar predicciones de salida.

- Implicaciones futuras:

Aunque los LLM tienen el potencial de revolucionar diversos sectores, es importante ser conscientes de sus limitaciones e implicaciones éticas. Las empresas y los trabajadores deben considerar detenidamente las compensaciones y los riesgos asociados al uso de los LLM, y los desarrolladores deben seguir perfeccionando estos modelos para minimizar los sesgos y mejorar su utilidad en diferentes aplicaciones. 

A continuación, veremos una de las principales limitaciones a tener en cuenta a la hora de utilizar modelos de lenguaje, y es una variable que depende de cada implementación en específico.

### Máximo número de tokens

El tamaño del contexto del LLM, o el número máximo de tokens que el modelo puede procesar, viene determinado por la implementación específica del LLM. En el caso de la implementación de OpenAI en LangChain, el número máximo de tokens viene definido por el modelo OpenAI que se esté utilizando. Para encontrar el número máximo de tokens para el modelo OpenAI, consulte el atributo max_tokens proporcionado en la documentación o API de OpenAI. 

OpenAI API Doc: https://platform.openai.com/docs/models/overview

Por ejemplo, si estás usando el modelo GPT-3, el número máximo de tokens que es soportado por el modelo son **2049** tokens. La máxima cantidad de tokens también depende de la versión específica lanzada del modelo (por ejemplo, davinci, curie, babbage o ada). Cada versión tiene diferentes limitaciones, normalmente las versiones más recientes son las que más número de tokens soportan.

Es importante asegurarse de que el texto de entrada no supera el número máximo de tokens admitidos por el modelo, ya que esto puede provocar errores durante el procesamiento. Para ello, puede dividir el texto de entrada en trozos más pequeños y procesarlos por separado (chunks), asegurándose de que cada chunk está dentro del límite de tokens permitido. 

Veamos un ejemplo de cómo tratar un texto que excede el límite máximo de tokens para un determinado LLM utilizando LangChain. Tenga en cuenta que el siguiente código es en parte pseudocódigo. No se supone que se ejecute, pero debería darte una idea de cómo manejar textos más largos que el límite máximo de tokens.

In [96]:
""" 
Funciones de utilidad para procesas texto y tokens en LLMs
"""
def split_text_into_chunks(input_text, max_tokens):
    chunks = len(input_text) // max_tokens    
    if (len(input_text) > max_tokens) and (len(input_text)%max_tokens != 0):
        process_chunks = [input_text[max_tokens*char:(char+1)*max_tokens] for char in range(chunks)] 
        process_chunks.append(input_text[max_tokens*chunks:len(input_text)])
        return process_chunks
    elif (len(input_text) > max_tokens) and (len(input_text)%max_tokens==0):
        return [input_text[max_tokens*char:(char+1)*max_tokens] for char in range(chunks)]
    return input_text

def combine_results(chunk_list):
    return "".join(chunk_list)

input_text = "Intenta escribir un input de texto muy largo para procesar"
max_tokens = 10
print(split_text_into_chunks(input_text, max_tokens))
print(combine_results(split_text_into_chunks(input_text, max_tokens)))

['Intenta es', 'cribir un ', 'input de t', 'exto muy l', 'argo para ', 'procesar']
Intenta escribir un input de texto muy largo para procesar


In [ ]:
from langchain.llms import OpenAI

# Antes de ejecutar el código, asegurar tener la OpenAI key
# en las variables de entorno "OPENAI_API_KEY"
# Instanciar el LLM
llm = OpenAI(engine="text-davinci-003")

# Input text
input_text = "Intenta escribir un input de texto largo para procesar"

# Determinamos el número máximo de tokens permitidos según la doc de OpenAI
max_tokens = 4097

# Divide el input de texto en chunks según el máximo de tokens
text_chunks = split_text_into_chunks(input_text, max_tokens)

# Procesar cada chunk de forma separada
results = []
for chunk in text_chunks:
    result = llm(chunk)
    results.append(result)

# Combinamos los resultados
final_result = combine_results(results)
final_result

**Distribuciones de Tokens y prediciendo el siguiente Token**

Los Large Language models, como GPT-3 y GPT-4, se entrenan previamente con grandes cantidades de datos de texto y aprenden a predecir el siguiente token de una secuencia basándose en el contexto proporcionado por los tokens anteriores. Los modelos de la familia GPT utilizan el modelado causal del lenguaje, que predice el siguiente token teniendo sólo acceso a los tokens anteriores. Este proceso permite a los LLM generar texto contextualmente relevante.

El siguiente código utiliza la clase OpenAI de LangChain para cargar la variación Davinci de GPT-3 utilizando la key text-davinci-003 para completar la secuencia, que da como resultado la respuesta. 

In [99]:
from langchain.llms import OpenAI
llm = OpenAI(
    engine="text-davinci-003",
    temperature=0
)
text = "Cuál sería un buen nombre para una empresa que haga IA generativa?"
print(llm(text))

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.




GenerAI Technologies.


**Trackeando el uso de Tokens**

Puedes utilizar la librería Langchain para trackear el uso de Tokens.

In [100]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
llm = OpenAI(
    engine="text-davinci-003",
    n=2,
    best_of=2
)
# Observamos el uso de tokens y su costo total de la API
with get_openai_callback() as cb:
    result = llm("Cuentame un chiste sobre IAs")
    print(cb)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


Tokens Used: 77
	Prompt Tokens: 10
	Completion Tokens: 67
Successful Requests: 1
Total Cost (USD): $0.0015400000000000001


### **Few-shot learning**

El few-shot learning es una capacidad notable que permite a los LLM aprender y generalizar a partir de ejemplos limitados. Las instrucciones sirven de entrada a estos modelos y desempeñan un papel crucial en la consecución de esta característica. Con LangChain, los ejemplos pueden quemarse, pero seleccionarlos dinámicamente a menudo resulta más potente, ya que permite a los LLM adaptarse y abordar rápidamente tareas con un mínimo de datos de entrenamiento.

Para ello se utiliza la clase FewShotPromptTemplate, que recibe una PromptTemplate y una lista de ejemplos de disparos. La clase formatea la plantilla con algunos ejemplos de tomas, lo que ayuda al LLM a generar una mejor respuesta. Podemos agilizar este proceso utilizando FewShotPromptTemplate de LangChain para estructurar el enfoque:

In [101]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

# Ejemplos
examples = [
    {
        "query": "Cómo está el clima?",
        "answer": "Están lloviendog atos y perros, mejor lleva sombrilla!"
    }, {
        "query": "Cuantos años tienes?",
        "answer": "Los años son los un número, yo no dependo del tiempo."
    }
]

# Creemos el Template de ejemplo
example_template = """
User: {query}
AI: {answer}
"""

# Ejemplo de prompt para el template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# Separamos el prompt en prefix y sufix, para usarlas como instrucciones
prefix = """
Los siguientes son extractos de conversaciones con un asistente de IA
AI. El asistente es conocido por su humor e ingenio, y ofrece
respuestas divertidas y entretenidas a las preguntas de los usuarios. He aquí algunos
ejemplos:
"""
suffix = """ 
User: {query}
AI: """

# Finalmente creamos el few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

Realizamos la inferencia mediante el Few-Shot learning:

In [103]:
from langchain.chat_models import ChatOpenAI 
from langchain import LLMChain

# Modelo
chat = ChatOpenAI(
    engine="gpt-35-turbo",
    temperature=0.0
)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("Cuál es el significado de la vida?")

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


'Esa es una pregunta profunda y filosófica. Pero si me preguntas a mí, diría que el significado de la vida es encontrar la felicidad y hacer del mundo un lugar mejor. Y, por supuesto, tener un buen sentido del humor también ayuda.'

### **Habilidades emergentes, leyes de escala y alucinaciones**

Otro aspecto de los LLM son sus capacidades emergentes, que surgen como resultado de un amplio preentrenamiento en vastos conjuntos de datos. Estas capacidades no se programan explícitamente, sino que surgen a medida que el modelo encuentra patrones en los datos. Los modelos LangChain aprovechan estas capacidades emergentes trabajando con varios tipos de modelos, como modelos de chat y modelos de embeddings de texto. Esto permite a los LLM realizar diversas tareas, desde responder preguntas hasta generar texto y ofrecer recomendaciones.

Por último, las leyes de escalado describen la relación entre el tamaño del modelo, los datos de entrenamiento y el rendimiento. En general, a medida que aumentan el tamaño del modelo y el volumen de datos de entrenamiento, también lo hace el rendimiento del modelo. Sin embargo, esta mejora está sujeta a rendimientos decrecientes y puede no seguir un patrón lineal. Es esencial tener un equilibrio entre el tamaño del modelo, los datos de entrenamiento, el rendimiento y los recursos empleados en el entrenamiento a la hora de seleccionar y ajustar los LLM para tareas específicas.

Los LLMs tienen capacidades notables, pero no están libres de defectos. Una limitación notable es la aparición de alucinaciones, en las que estos modelos producen texto que parece confiable a primera vista, pero que en realidad es incorrecto o no guarda relación con la entrada dada. Además, los LLM pueden presentar sesgos derivados de sus datos de entrenamiento, lo que puede perpetuar estereotipos o generar resultados no deseados.

**Ejemplos con Prompts sencillos (multitask): Text Summarization, Text Translation y Question Answering**

Llamando modelos OpenSource de HuggingFace:

In [104]:
#!pip install -q huggingface_hub

In [105]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=["question"]
)
question = "Cual es la capital de Colombia?"

In [ ]:
from langchain import HuggingFaceHub, LLMChain

# LLM Hub
hub_llm = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={'temperature':0}
)

# Prompt template
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)
print(llm_chain.run(question))

También podemos modificar el prompt template para incluir múltiples preguntas:


In [106]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)
print(res)

In [ ]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
	"What color is a ripe banana?\n"
)
llm_chain.run(qs_str)

**Text Summarization**

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(engine="gpt-3.5-turbo", temperature=0)
summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)
text = "LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."
summarized_text = summarization_chain.predict(text=text)

**Text Translation**

In [ ]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], template=translation_template)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)
source_language = "English"
target_language = "French"
text = "Your text here"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)